# Cutting plane method aka localisatoin method

Really an alternative to subgradient method. Generally a bit more expensive but in some cases (which?) can work better. Can handle differentiable and non-differentiable problems providing the oracle is implemented to return the subgradient rather than gradient. 

Begin by initalising an LP and solving with CVXPY 

In [5]:
import numpy as np
import cvxpy as cp

m = 100 # num instances 
n = 20 # num variables

A = np.random.normal(0,1,size=(m,n))
b = np.random.uniform(0,1,size=(m))
c = -A.T @ np.random.uniform(0,1,size=(m)) # so problem instance is bounded 
print(A.shape, b.shape, c.shape)


x = cp.Variable(n)
obj = cp.Minimize(c.T @ x)
constraints = [A @ x <= b]
problem = cp.Problem(obj, constraints)
problem.solve()
optimal_point = x.value
optimal_point
problem.value

(100, 20) (100,) (20,)


-4.20848956205706

In [10]:
def piecewise_affine_parameters(m=100, n=20):
    A = np.random.normal(0,1,size=(m,n))
    b = np.random.uniform(0,1,size=(m))
    return A,b

def piecewise_affine_func(x):
    A, b = picewise_affine_func_params
    return np.max(A @ x + b)

picewise_affine_func_params = piecewise_affine_parameters()


In [14]:
def subgradient_of_piecewise_affine(x):
    # from https://math.stackexchange.com/questions/2088764/how-to-calculate-set-of-subgradients
    A,b = picewise_affine_func_params
    f_x = piecewise_affine_func(x)
    num_rows, num_cols = A.shape
    for i in range(num_rows):
        if A[i] @ x + b == f_x:
            return A[i]

def cutting_plane_oracle(x):
    if x == optimal_point:
        return True, None 
    else:
        # evalueate sub gradient 
        subgradient = subgradient_of_piecewise_affine(A,b,f_x)
        return False, subgradient 
    
def maximum_volume_ellipsoid(polyhedron):
    '''
    Given polyhedron, find the maximum volume ellipsoid. This is 
    the largest ellipsoid contained within the polyhedron. 
    
    Returns center of this ellipsoid 
    
    
    Size of ellipsoid is proportional to det B 
    
    the square roots of the eigenvalues of B are the lengths of the semi-axes of the ellipsoid 
    '''
    B = cp.Variable((n,n))
    d = cp.Variable(n)
    
    contraints = []
    for linear_constraint in polyhedron:
        a,b = linear_constraint
        contraints.append(a.T @ d <= b)
    
    obj = cp.Minimize(cp.log_det(B))
    problem = cp.Problem(obj, constraints)
    problem.solve()
    
    # B defines the ellisoid, take the center
    
    
    
    
    
    

In [15]:
# linear const : [a in Rn, b in Rn] 
polyhedron = [] # list of linear constraints. 


k = 0 
for _ in range(2):
    # choose a point x(k+1) in polydreon
    x_k1 = maximum_volume_ellipsoid(polyhedron)
    
    # query the cutting-plane oracle 
    stop, subgradient = cutting_plane_oracle(x_k1)
    
    if stop:
        break
    else:
        polyhedron.append([subgradient, ])
        
    k+=1 

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP, even though each sub-expression is.
You are trying to minimize a function that is concave.